In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords

In [2]:
unida = pd.read_json('C:/Users/enterprise/Google Drive/fake_news_resources/raw_data/nacion_unida.json')
verif = pd.read_json('C:/Users/enterprise/Google Drive/fake_news_resources/raw_data/verificado_2018.json')

In [3]:
unida['title_list'] = unida['title'].apply(lambda x: str(x).split(' '))
unida['filter_title'] = unida['title_list'].apply(lambda x: [word for word in x if word not in stopwords.words('spanish')])

In [4]:
verif['title_list'] = verif['title'].apply(lambda x: str(x).split(' '))
verif['filter_title'] = verif['title_list'].apply(lambda x: [word for word in x if word not in stopwords.words('spanish')])

In [5]:
unida['filter_title_join'] = unida['filter_title'].apply(lambda x: ' '.join(x))
verif['filter_title_join'] = verif['filter_title'].apply(lambda x: ' '.join(x))

In [6]:
unida_str = unida['filter_title_join'].tolist()
verif_str = verif['filter_title_join'].tolist()

In [7]:
corpus = unida_str + verif_str
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(corpus)
names = vectorizer.get_feature_names()

In [8]:
mat_uni = vectorizer.transform(unida_str)
mat_ver = vectorizer.transform(verif_str)

In [9]:
df_mat_uni = pd.DataFrame(mat_uni.todense(), columns=names)
df_mat_ver = pd.DataFrame(mat_ver.todense(), columns=names)

In [10]:
lst_mat_uni = df_mat_uni.values.tolist()
lst_mat_ver = df_mat_ver.values.tolist()

In [11]:
def l2_norm(a):
    return(math.sqrt(np.dot(a, a)))

In [12]:
def cosine_similarity(a, b):
    return(np.dot(a,b) / (l2_norm(a) * l2_norm(b)))

In [13]:
maybe = []

for i in range(len(lst_mat_ver)):
    aux = []
    for j in range(len(lst_mat_uni)):
        sim = cosine_similarity(np.array(lst_mat_uni[j]), np.array(lst_mat_ver[i]))
        aux.append(sim)
    maybe.append(aux)

In [14]:
max_maybe = []

for com in maybe:
    max_maybe.append(max(com))

In [15]:
index_maybe = []

for r in range(len(maybe)):
    index = [i for i, j in enumerate(maybe[r]) if j == max_maybe[r]]
    index_maybe.append(index)

In [16]:
new_index = []

for index in index_maybe:
    if len(index) == 1:
        new_index.append(index[0])
    else:
        #print(index)
        pass

In [17]:
class_est = []

for i in range(unida.shape[0]):
    if i in new_index:
        class_est.append(1)
    else:
        class_est.append(0)

In [18]:
df_class = pd.DataFrame({'class': class_est})

In [19]:
result = pd.concat([unida, df_class], axis=1, sort=False)

In [20]:
result_c1 = result[result['class'] == 1]

In [23]:
print(result_c1.loc[[931]])

                                                  text  \
931   Militante рor años del PAN, exregidor en Mont...   

                                                 title  \
931  anaya paga 4 millones 640 mil pesos ecuestador...   

                                            title_list  \
931  [anaya, paga, 4, millones, 640, mil, pesos, ec...   

                                          filter_title  \
931  [anaya, paga, 4, millones, 640, mil, pesos, ec...   

                                     filter_title_join  class  
931  anaya paga 4 millones 640 mil pesos ecuestador...      1  


In [22]:
result.to_json('cross_title.json', orient='records')